In [1]:
from torch_geometric.datasets import Planetoid
import torch

In [2]:
dataset = Planetoid(
    root="/home/ray/code/python/python_data_course/机器学习与深度学习导论/data", name="Cora"
)
dataset

Cora()

In [4]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

In [5]:
class GraphSageNet(nn.Module):
    def __init__(self):
        super(GraphSageNet, self).__init__()
        self.conv1 = SAGEConv(dataset.num_node_features, 16)
        self.conv2 = SAGEConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.softmax(x, dim=1)
        return x

In [7]:
model = GraphSageNet()
model

GraphSageNet(
  (conv1): SAGEConv(1433, 16, aggr=mean)
  (conv2): SAGEConv(16, 7, aggr=mean)
)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
data = dataset[0].to(device)

In [9]:
import torch.optim as optim

In [11]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [12]:
model.train()
for epoch in range(200):
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    _, pred = torch.max(out[data.train_mask], dim=1)
    correct = (pred == data.y[data.train_mask]).sum().item()
    acc = correct/data.train_mask.sum().item()
    print('Epoch {:03d} train_loss: {:.4f} train_acc: {:.4f}'.format(epoch, loss.item(), acc))

Epoch 000 train_loss: 1.9451 train_acc: 0.1286
Epoch 001 train_loss: 1.9237 train_acc: 0.3714
Epoch 002 train_loss: 1.8896 train_acc: 0.5500
Epoch 003 train_loss: 1.8303 train_acc: 0.6071
Epoch 004 train_loss: 1.7631 train_acc: 0.6714
Epoch 005 train_loss: 1.6962 train_acc: 0.7357
Epoch 006 train_loss: 1.6212 train_acc: 0.7429
Epoch 007 train_loss: 1.5604 train_acc: 0.8429
Epoch 008 train_loss: 1.5042 train_acc: 0.9143
Epoch 009 train_loss: 1.4347 train_acc: 0.9214
Epoch 010 train_loss: 1.3956 train_acc: 0.9500
Epoch 011 train_loss: 1.3594 train_acc: 0.9500
Epoch 012 train_loss: 1.3364 train_acc: 0.9714
Epoch 013 train_loss: 1.3175 train_acc: 0.9429
Epoch 014 train_loss: 1.2701 train_acc: 0.9714
Epoch 015 train_loss: 1.2644 train_acc: 0.9714
Epoch 016 train_loss: 1.2396 train_acc: 0.9786
Epoch 017 train_loss: 1.2266 train_acc: 0.9857
Epoch 018 train_loss: 1.2243 train_acc: 1.0000
Epoch 019 train_loss: 1.2020 train_acc: 0.9929
Epoch 020 train_loss: 1.2011 train_acc: 1.0000
Epoch 021 tra

In [13]:
model.eval()
out = model(data)
loss = criterion(out[data.test_mask], data.y[data.test_mask])
_, pred = torch.max(out[data.test_mask], dim=1)
correct = (pred == data.y[data.test_mask]).sum().item()
acc = correct/data.test_mask.sum().item()
print("test_loss: {:.4f} test_acc: {:.4f}".format(loss.item(), acc))

test_loss: 1.3794 test_acc: 0.8120
